<a href="https://colab.research.google.com/github/VladGrigoras/CNS/blob/main/CNS_Lab3_RingNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
sns.set()

# global defaults for plots - optional
sns.set_theme(style="ticks",
              palette="colorblind",
              font_scale=1.7,
              rc={
              "axes.spines.right": False,
              "axes.spines.top": False,
          },
          )

# Ben-Yishai Ring Network

## Introduction 
The aim of this lab is to implement the ring network model which was originally proposed by [Ben-Yishai et al. (1995)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC42058/pdf/pnas01493-0220.pdf) to explain orientation tuning of neurons in visual cortex. As discussed in the lectures, this model exhibits distinct activity regimes in which orientation tuning 
is determined by:

1. The tuning of feedforward inputs to the neurons.
2. Recurrent synapses within the V1 circuit.

Due to the behaviour of the ring model in the recurrent-dominated regime, it has since been adopted as a model for persistent activity of neurons observed in brain areas such as prefrontal cortex during the delay period of working memory tasks. 

In the following exercises you will implement the model as a numerical simulation, vary the parameters in order to simulate the
different regimes of the model, explore the behaviour of the model in each regime and think about possible implications of your findings for orientation tuning in visual cortex and for working memory in prefrontal cortex.

## Model
### Continuous neuron
The original model consists of a continuous ring of rate neurons which are parameterised by their preferred orientation $\theta\in (-\pi/2,\pi/2)$, as a featureless bar can only have distinguishable orientations in the range $-90^\circ$ to $+90^\circ$.

The dynamics of the model is defined as,

$$
\begin{aligned}
\tau \frac{dr (\theta, t)}{dt}&=-r(\theta, t) + [h(\theta, t)]_{+}  \hspace{1cm}(1)
\end{aligned}
$$

where $[x]_{+}=x$ if $x>0$ and $0$ otherwise, that is the ReLU activation function. The net input is,

$$
\begin{aligned}
h(\theta, t) &= u(\theta-\theta_s)+\int_{-\pi/2}^{\pi/2} W(\theta,\theta')r(\theta',t)d\theta ' \hspace{1cm}(2)
\end{aligned}
$$

where $\theta_s$ is the stimulus orientation. The recurrent weights are,

$$
\begin{aligned}
W(\theta,\theta') &= A + B \cos(2(\theta-\theta'))
\end{aligned}
$$

and the feedforward external input,

$$
\begin{aligned}
u(\theta-\theta_s) = C + D \cos(2(\theta-\theta_s)) = c[1-\epsilon+\epsilon\cos(2(\theta-\theta_s))]
\end{aligned}
$$

where the second parameterisation of the input is interpreted as a stimulus contrast parameter $c$ and tuning strength parameter $\epsilon$.

### Discretisation
This formulation of the model as a continuous network was chosen by Ben-Yishai et al. because it can be solved analytically to find the steady state responses. However, for running simulations we need to convert the model into a network of N discrete neurons as follows: 

$$
\begin{aligned}
\tau \frac{dr_i (t)}{dt}&=-r_i(t) + [h_i(t)]_{+}
\\h_i(t) &= u_i+\frac{1}{N}\sum_{j=1}^N W_{ij}r_j(t)
\\W_{ij}&=W_0 + W_1\cos (2(\theta_i-\theta_j))
\\u_i&=c[1-\epsilon + \epsilon \cos(2(\theta_i-\theta_s))]
\end{aligned}
$$

where neurons are indexed over a discrete set of angles $\theta_i=i\pi/N-\pi/2$.


#Implementation of the System
This section is dedicated to implementing the dynamical system and how to evaluate it.

##Exercise 1 $-$ Implementation of Functions

Write code to implement the right hand side of each of the above equations.

In [ ]:
#Complete the functions:

def generate_u(N, Nt, c, epsilon, theta, theta_s): 
  '''
  inputs
  N: number of neurons
  Nt: number of time steps 
  c, epsilon: input parameters
  theta: N dimensional ndarray of preferred orientations
  thea_s: stimulus orientation

  output
  u: NtxN dimensional ndarray of inputs across time
  '''

  u = np.zeros([Nt,N])
  
  return '...'

def h(r,u,W,t):
  '''
  inputs
  r: N dimensional ndarray 
  u: NtxN dimensional ndarray
  W: NxN dimensional ndarray
  t: time point

  output
  h: N dimensional ndarray
  '''

  return '...'

def dr_dt(r,u,W,tau,t):
  '''
  inputs
  r: N dimensional ndarray 
  u: NtxN dimensional ndarray
  W: NxN dimensional ndarray
  tau: time constant
  t: time point

  output
  dr_dt: N dimensional ndarray
  '''

  return '...'


def generate_W(N, W0, W1, theta): 
  '''
  inputs
  N, W0, W1: model parameters (int)
  theta: N dimensional ndarray

  output
  W: NxN dimensional ndarray
  '''
  
  W = np.zeros([N,N])

  return '...'

## Evaluating the system
To simulate the dynamics in time, we will use the Euler method which we have implemented below. It works for any dynamical system that can be represented as $\dot{\bf{x}}=f(\bf{x},t)$.

In [ ]:
def euler(f,x0,Nt,dt):
  """
  in:
  f : RHS of the differential equation (dx(x)/dt), takes x as an input
  x0 : initial state
  dt : step size
  Nt : number of steps

  out:
  xt : an Nt+1 x len(x) matrix of the state of the system over time
  """

  xt = np.zeros([Nt+1, len(x0)])
  xt[0] = x0

  for i in range(1,Nt+1):
    xt[i] = xt[i-1]+dt*f(xt[i-1],i-1)

  return xt

## Wrapper functions
We define a function to simulate one run for the Ben-Yishai ring network with a time-dependent input, using the Euler method and starting from an initial condition $r_{init}$.

In [ ]:
def BY_ring_network(r_init, W, u, tau, Nt, dt): 

    def F(r,i):
      
      return dr_dt(r,u,W,tau,i)

    rt = euler(F,r_init,Nt,dt)
    
    return rt

# Simulation with Different Regimes

To get a first impression for how the networks behave in their default settings, we next simulate and plot the activity of the network in three different regimes. 

### Default parameters
As default parameters we set:


*  $N=100$ the number discrete neurons
*  $\tau=10$ms the time constant of the network
*  $\theta_s = 0$ the stimulus orientation
*  $c=0.5$ the external input strength (or stimulus contrast)
*  $\Delta t=1$ the time discretisation

The values for $\epsilon,W_0,W_1$ depend on the simulated regime and are defined in subsequent sections.

In [ ]:
N = 100
tau_m = 10
theta_s = 0
c = 0.5
dt = 1

theta_p = np.linspace(-np.pi/2, np.pi/2, N+1)[:N]

## Exercise 2.1 $-$ Hubel and Wiesel regime
In the Hubel and Wiesel regime, the network is driven by strongly tuned feedforward inputs and there is no recurrent connectivity. To model this, set $W_0$ = 0, $W_1 = 0$ so that $h_i (t) = u_i$. As the input is strongly tuned, $\epsilon$ is high. Set $\epsilon$ = 1. 

1. What do you expect the steady state to look like in this regime? Write the steady state equation for $r_i$ from equation (1) and sketch the response you expect to the inputs $u_i$.

Then, run the cell below and compare your prediction with the numerical results.

In [ ]:
Nt = 500
epsilon = 1
W_HW = generate_W(N, 0, 0, theta_p)
u = generate_u(N, Nt, c, epsilon, theta_p, theta_s)

output = BY_ring_network(np.zeros(N), W_HW, u, tau_m, Nt, dt)

fig, ax = plt.subplots(1,2,  figsize = (20,5))
sns.heatmap(output.T, ax = ax[0], xticklabels = 100, yticklabels = 20)
ax[0].set_yticklabels(ax[0].get_yticklabels(), rotation=0)
ax[0].set_title('Network output')
ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('Neuron')

ax[1].plot(np.degrees(theta_p), output[-1], label = '$r_i$')
ax[1].plot(np.degrees(theta_p), u[-1], label = '$u_{i}$')
ax[1].set_title('Steady state response')
ax[1].set_ylabel('Network output')
ax[1].set_xlabel('Preferred orientation ($^\circ$)')
ax[1].legend(bbox_to_anchor=(1., 1.0), loc='upper left', fontsize = 20)
plt.show()

## Exercise 2.2 $-$ Uniform Inhibition regime

The uniform inhibition regime was not discussed in lectures, but was included in the original paper of Ben-Yishai et al. This regime is defined by the uniform mutual inhibition between all neurons. This means that the recurrent weights are negative and do not differ around the ring. Therefore, set $W_0 = −1$ and $W_1 = 0$. 

1. As an exercise plot 1) the response at steady state and 2) the feedforward input $u_i$, for values of $\epsilon$ as low as 0.01 and as high as 1. 
2.   How does $\epsilon$ influence the results? Why do you think this happens?
3.   For the experiments section below, pick $\epsilon = 1$ when simulating the non-uniform regime (you can also explore the consequences of varying it if you like).

In [ ]:
Nt = 500
epsilons = np.linspace(0.01, 1, 4)   # try a range of different values for the input tuning parameter

# Complete with the appropriate inputs to the function
W_UI = generate_W(...)

# Your solution here:

## Exercise 2.3 $-$ Marginal Regime

The above two regimes assume that the external input is strongly tuned and that recurrent input is either absent or uniform. Another theory is that the tuning of the inputs themselves is very weak, but is amplified by non-uniform interactions within the network. This regime is called the recurrent-dominated regime, or the “marginal” regime, which is a terminology inherited from models for spontaneous symmetry breaking in statistical physics. 

1. To model this case, set the external input tuning $\epsilon$ to be small (e.g. 0.01), and set the connections to be strongly dependent on the preferred orientations of their pre- and post-synaptic
neurons by choosing $W_0 = −1$, $W_1 = 3$. Simulate and plot as above.

2. Try varying $\epsilon$ to even smaller values (or larger ones), and investigate how this influences the profile of the network response and the input profile $u_i$.

In [ ]:
Nt = 500
epsilons = np.linspace(0.01, 1, 4) #try varying this

# Complete with the appropriate inputs to the function
W_MR = generate_W(...) 

# Your solution here:

## Compare weight matrices

To visualise how the weight matrices look like for the different regimes, we plotted them next to each other below*: the marginal regime is the only regime where the weight matrix shows any structure.  

*Note that the code below will work only if the variables names `W_HW`, `W_UI`, `W_MR` were left unchanged in the previous cells

In [ ]:
Ws = [W_HW, W_UI, W_MR]
regimes = ['Hubel and Wiesel', 'Uniform inhibition', 'Marginal Regime']

fig, ax = plt.subplots(1,len(Ws), figsize = (20,5))

for i in range(len(Ws)):
  ax[i] = sns.heatmap(Ws[i], vmin = -4, vmax = 2, ax = ax[i], xticklabels = 20, yticklabels = 20)
  ax[i].set_yticklabels(ax[i].get_yticklabels(), rotation=0)
  ax[i].set_ylabel('Neuron')
  ax[i].set_xlabel('Neuron')
  ax[i].set_title(regimes[i])
plt.show()

# Experiments

The following experiments are designed to reveal how the behaviour of the ring network differs between each of the above regimes. Run each experiment for the Hubel and Wiesel, uniform inhibition and marginal regime. For experiments involving changes in stimuli (rotation or deletion), make sure that the network has settled/stabilised prior to changing the stimulus.

## Exercise 3.1 $-$ Stimulus Rotation

* Once the activity has settled under the presentation of a stimulus at $\theta_s = 0$, rotate the stimulus by $60$ degrees. 
  1. How do you expect the activity to change after the rotation?
  2. How do you think the change of activity depends on the network regime? 
  3. Do you think the behaviour would differ if you rotate the stimulus
by different angles?

* Examine the time course of the network activity in response to this stimulus rotation and compare the simulations with your predictions.

There are two different possibilities on how to use the function BY_ring_network() to collect the results of a stimulus rotation. You can either define your stimulus for one whole simulation, so that the stimulus in itself already is changing (as shown below). Or you can run two seperate simulations with two different stimuli and use the last timepoint of the 1st simulation (using the 1st stimulus) as `r_init` for the second simulation (using the 2nd stimulus), as shown below for the stimulus deletion experiments. 

In [ ]:
Nt = 5000

epsilon_HW = 1
epsilon_UI = 1
epsilon_MR = 0.01

theta_s_1sthalf = 0
theta_s_2ndhalf = np.pi/3 # 60 degrees rotation

epsilons = [epsilon_HW, epsilon_UI, epsilon_MR]
Ws = [W_HW, W_UI, W_MR]
network_names = ['Hubel and Wiesel regime', 'Uniform Inhibition', 'Marginal Regime']

# Your solution here:

## Exercise 3.2 $-$ Stimulus Deletion

We next present a stimulus with orientation $\theta_s = 0$. Once the activity has settled, the stimulus is deleted by setting $u_i = c$. 

1. How does the response following the deletion of the stimulus depend on the regime of the model? 
2. In which network regime could you decode the stimulus orientation after the stimulus has been deleted?

In [ ]:
Nt = 5000

epsilon_HW = 1
epsilon_UI = 1
epsilon_MR = 0.01


theta_s = 0

epsilons = [epsilon_HW, epsilon_UI, epsilon_MR]
Ws = [W_HW, W_UI, W_MR]
network_names = ['Hubel and Wiesel regime', 'Uniform Inhibition', 'Marginal Regime']

# Your solution here:

# Noisy Inputs

As discussed in the lectures, neural activity is noisy. For the following experiments the total background noise is modelled as an additive Gaussian variable. Therefore we define $u_i (t)$ as:

$$
\begin{aligned}
u_i(t)&=c[1-\epsilon+\epsilon \cos(2\theta)]+\sigma \eta_i(t)
\end{aligned}
$$

$$
\begin{aligned}
\eta_i(t) \sim \mathcal{N}(0,1)
\end{aligned}
$$

with $\sigma$ the magnitude of the noise (remember that this depends on the discretisation timestep when numerically integrating). To simulate, draw $\eta_i$ independently for each neuron at each timestep.

## Exercise 4.1 $-$ Persistent Noisy Input

Simulate all three model regimes with the newly defined noisy input $u_i (t)$, and with different choices for $\sigma$. 
1. How does the noise change the responses of the networks? 
2. Is there a network which is more robust to noise than the others? 
3. Try simulating the marginal regime but now with completely untuned inputs ($\epsilon = 0$). How does noise influence the response of the network? 
4. Compare with noise values ranging from $\sigma = 0.05$ to $σ = 1$. What do you think is happening? How does increasing $\epsilon$ change the results?

In [ ]:
def generate_noisy_u(N, Nt, c, epsilon, theta, theta_s, sigma, rseed = 1, dt = 1): 
    
    u = np.zeros([Nt,N])
    rng = np.random.RandomState(seed = rseed)
    
    #Complete the code here:
    u[:] = '...'
    
    return u

In [ ]:
epsilon_HW = 1
epsilon_UI = 1
epsilon_MR = 0.01

theta_s = 0

epsilons = [epsilon_HW, epsilon_UI, epsilon_MR, 0]
Ws = [W_HW, W_UI, W_MR, W_MR]

network_names = ['Hubel and Wiesel regime', 'Uniform Inhibition', 'Marginal Regime', 'Marginal Regime - untuned input']

sigmas = np.linspace(0.05, 1, 4)

# Your solution here:

## Exercise 4.2 $-$ Noisy Input After Stimulus Deletion

Next we repeat the stimulus-deletion experiment, but this time include the persistent noisy input described above. Once the stimulus is deleted, the external input should be $u_i (t) = c+\eta_i (t)$. 

1. What happens in each regime? 
2. How would the noisy input influence the ability to decode the stimulus orientation after the stimulus has been deleted? 
3. How might this be relevant when modelling working memory?

In [ ]:
def generate_untuned_noisy_u(N, Nt, c, sigma, rseed = 1, dt = 1): 
    
    u  = np.zeros([Nt,N])
    rng = np.random.RandomState(seed = rseed)
    
    #Complete the code for the untuned input (after stimulus deletion)
    u[:] = '...'
    
    return u

In [ ]:
epsilon_HW = 1
epsilon_UI = 1
epsilon_MR = 0.01

sigma = 0.2

theta_s = 0
network_names = ['Hubel and Wiesel regime', 'Uniform Inhibition', 'Marginal Regime']

epsilons = [epsilon_HW, epsilon_UI, epsilon_MR]
Ws = [W_HW, W_UI, W_MR]

# Your solution here:

# Bonus Section: Derivation of the Tuning Curves (Optional)

In this section, we will derive the expression for the tuning curve of the neurons in the ring network. 

We consider the discrete version of the equations and the case where the stimulus is $\theta_s=0$ to simplify notation. The steady state of (the discretised) equation (1) is $r^*_i$, which represents the activity profile of the network relative to the orientation of the stimulus. Thus, it also represents the tuning curve of a neuron, centred at its preferential orientation.

The solution for the steady state is given by the equation $r^*_i = [h^*_i]_+$, where $h^*_i$ is given by equation (2) and is in the form:

$$
\begin{aligned}
h^*_i &= c[1-\epsilon + \epsilon \cos(2\theta_i)] + \frac{1}{N}\sum_{j=1}^N [W_0 + W_1\cos(2(\theta_i-\theta_j)]r^*_j \hspace{1cm}(S1)\\
\end{aligned}
$$

To find the solution for $r^*_i$, we need to move to the Fourier space. For an even function $r_j$ with $j\in[1,N]$ (we drop the $^*$ notation for now), we can write $r_j$ as a Fourier series of cosines:

$$
\begin{aligned}
r_j &= \sum_{k=0}^{N-1} \hat{r}_k \cos\left(\frac{2\pi}{N}kj\right),  \hspace{1cm}(S2)
\end{aligned}
$$

where $\hat{r}_k$ are the Fourier coefficients of $r_j$. Note that the following orthogonality relation holds:

$$
\begin{aligned}
\frac{1}{N}\sum_{j=1}^{1}\cos\left(\frac{2\pi}{N}kj\right)\cos\left(\frac{2\pi}{N}k'j\right) = \delta_{k,k'}. \hspace{1cm}(S3)
\end{aligned}
$$

1. Substitute eq. (S2) into eq. (S1) to simplify it in terms of Fourier coefficients. [Use the property in eq (S3) and note that $\theta_j = j2\pi/N$]


2. Now write the full equation for the steady state by substituing the previous equation for $h^*_i$ into $r^*_i = [h^*_i]_+$ and finding an equation for each coefficient. [Note that you will have to expand $r^*_i$ into the Fourier series]


# References
[Ben-Yishai, R., Bar-Or, R. L., & Sompolinsky, H. (1995). Theory of orientation tuning in visual cortex. Proceedings of the National Academy of Sciences, 92(9), 3844-3848. ](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC42058/)